# Ajuste de Hiperparâmetros e Seleção de Features

Este notebook demonstra como aplicar técnicas de otimização de modelos de machine learning:

1. **Grid Search** para ajuste de hiperparâmetros no modelo de árvore de decisão para classificação (California Housing)
2. **Seleção de Features** para identificar características importantes na previsão de diabetes (Pima Indians Diabetes)

## Objetivos:
- Aprender a usar Grid Search para encontrar os melhores hiperparâmetros
- Entender como identificar as features mais relevantes
- Comparar o desempenho antes e depois da otimização


In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, mutual_info_classif, mutual_info_regression
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Configuração para visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)


## Parte 1: Grid Search para Classificação - California Housing

Vamos trabalhar com o dataset California Housing, mas transformando-o em um problema de classificação criando categorias de preços de casas.


In [ ]:
# Carregando o dataset California Housing
california_housing = fetch_california_housing()
X_california = pd.DataFrame(california_housing.data, columns=california_housing.feature_names)
y_california = california_housing.target

print("Dataset California Housing:")
print(f"Forma dos dados: {X_california.shape}")
print(f"Features: {list(X_california.columns)}")
print(f"Primeiras linhas:")
print(X_california.head())

# Transformando em problema de classificação
# Criando categorias baseadas nos quartis dos preços
y_california_cat = pd.cut(y_california, bins=4, labels=['Muito Baixo', 'Baixo', 'Alto', 'Muito Alto'])
print(f"\nDistribuição das categorias de preço:")
print(y_california_cat.value_counts())


In [ ]:
# Preparando os dados para classificação
X_train_cal, X_test_cal, y_train_cal, y_test_cal = train_test_split(
    X_california, y_california_cat, test_size=0.2, random_state=42, stratify=y_california_cat
)

print(f"Conjunto de treino: {X_train_cal.shape}")
print(f"Conjunto de teste: {X_test_cal.shape}")

# Modelo base (sem ajuste de hiperparâmetros)
dt_base = DecisionTreeClassifier(random_state=42)
dt_base.fit(X_train_cal, y_train_cal)

# Avaliando o modelo base
y_pred_base = dt_base.predict(X_test_cal)
accuracy_base = dt_base.score(X_test_cal, y_test_cal)

print(f"\n=== MODELO BASE (Árvore de Decisão) ===")
print(f"Acurácia: {accuracy_base:.4f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test_cal, y_pred_base))


### Aplicando Grid Search para Otimização de Hiperparâmetros

Agora vamos usar Grid Search para encontrar os melhores hiperparâmetros para nossa árvore de decisão.


In [ ]:
# Definindo o grid de hiperparâmetros para testar
param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}

print("Grid de hiperparâmetros definido:")
for param, values in param_grid.items():
    print(f"{param}: {values}")

# Criando o modelo para Grid Search
dt_grid = DecisionTreeClassifier(random_state=42)

# Aplicando Grid Search com validação cruzada
print(f"\nIniciando Grid Search...")
grid_search = GridSearchCV(
    estimator=dt_grid,
    param_grid=param_grid,
    cv=5,  # 5-fold cross validation
    scoring='accuracy',
    n_jobs=-1,  # Usar todos os cores disponíveis
    verbose=1
)

# Treinando o modelo com Grid Search
grid_search.fit(X_train_cal, y_train_cal)

print(f"\nGrid Search concluído!")
print(f"Melhores parâmetros encontrados:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"Melhor score (CV): {grid_search.best_score_:.4f}")


In [ ]:
# Avaliando o modelo otimizado
best_model = grid_search.best_estimator_
y_pred_optimized = best_model.predict(X_test_cal)
accuracy_optimized = best_model.score(X_test_cal, y_test_cal)

print(f"\n=== MODELO OTIMIZADO (Grid Search) ===")
print(f"Acurácia: {accuracy_optimized:.4f}")
print(f"Melhoria: {accuracy_optimized - accuracy_base:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test_cal, y_pred_optimized))

# Matriz de confusão
plt.figure(figsize=(10, 6))
cm = confusion_matrix(y_test_cal, y_pred_optimized)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=best_model.classes_, yticklabels=best_model.classes_)
plt.title('Matriz de Confusão - Modelo Otimizado')
plt.ylabel('Valores Reais')
plt.xlabel('Valores Previstos')
plt.show()


In [ ]:
# Visualizando a importância das features no modelo otimizado
feature_importance = pd.DataFrame({
    'feature': X_california.columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Importância das Features - Modelo Otimizado')
plt.xlabel('Importância')
plt.tight_layout()
plt.show()

print("Importância das features:")
print(feature_importance)


## Parte 2: Seleção de Features para Regressão - Pima Indians Diabetes

Agora vamos trabalhar com o dataset Pima Indians Diabetes para demonstrar técnicas de seleção de features em um problema de regressão.


In [ ]:
# Carregando o dataset Pima Indians Diabetes
# URL do dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
                'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

try:
    # Tentando carregar do URL
    df_pima = pd.read_csv(url, names=column_names)
    print("Dataset carregado com sucesso do URL!")
except:
    # Se não conseguir carregar do URL, criamos dados sintéticos baseados no dataset original
    print("Criando dados sintéticos baseados no dataset Pima Indians Diabetes...")
    np.random.seed(42)
    n_samples = 768
    
    # Gerando dados sintéticos baseados nas características do dataset original
    df_pima = pd.DataFrame({
        'Pregnancies': np.random.poisson(3.8, n_samples),
        'Glucose': np.random.normal(120.9, 32.0, n_samples),
        'BloodPressure': np.random.normal(69.1, 19.4, n_samples),
        'SkinThickness': np.random.normal(20.5, 16.0, n_samples),
        'Insulin': np.random.normal(79.8, 115.2, n_samples),
        'BMI': np.random.normal(32.0, 7.9, n_samples),
        'DiabetesPedigreeFunction': np.random.exponential(0.5, n_samples),
        'Age': np.random.normal(33.2, 11.8, n_samples),
        'Outcome': np.random.binomial(1, 0.35, n_samples)
    })
    
    # Ajustando valores negativos
    df_pima['Glucose'] = np.maximum(df_pima['Glucose'], 0)
    df_pima['BloodPressure'] = np.maximum(df_pima['BloodPressure'], 0)
    df_pima['SkinThickness'] = np.maximum(df_pima['SkinThickness'], 0)
    df_pima['Insulin'] = np.maximum(df_pima['Insulin'], 0)
    df_pima['BMI'] = np.maximum(df_pima['BMI'], 0)
    df_pima['Age'] = np.maximum(df_pima['Age'], 0)

print(f"\nDataset Pima Indians Diabetes:")
print(f"Forma dos dados: {df_pima.shape}")
print(f"Primeiras linhas:")
print(df_pima.head())

print(f"\nInformações do dataset:")
print(df_pima.info())

print(f"\nEstatísticas descritivas:")
print(df_pima.describe())


In [ ]:
# Transformando em problema de regressão
# Vamos usar as features para prever o valor de Glucose (como se fosse um problema de regressão)
X_pima = df_pima.drop(['Outcome', 'Glucose'], axis=1)  # Removendo Outcome e Glucose
y_pima = df_pima['Glucose']  # Usando Glucose como variável alvo

print(f"Features para regressão: {list(X_pima.columns)}")
print(f"Variável alvo: Glucose")
print(f"Forma dos dados: X={X_pima.shape}, y={y_pima.shape}")

# Dividindo os dados
X_train_pima, X_test_pima, y_train_pima, y_test_pima = train_test_split(
    X_pima, y_pima, test_size=0.2, random_state=42
)

print(f"\nConjunto de treino: {X_train_pima.shape}")
print(f"Conjunto de teste: {X_test_pima.shape}")

# Modelo base (sem seleção de features)
rf_base = RandomForestRegressor(random_state=42, n_estimators=100)
rf_base.fit(X_train_pima, y_train_pima)

# Avaliando o modelo base
y_pred_base_pima = rf_base.predict(X_test_pima)
mse_base = mean_squared_error(y_test_pima, y_pred_base_pima)
r2_base = r2_score(y_test_pima, y_pred_base_pima)

print(f"\n=== MODELO BASE (Random Forest) ===")
print(f"MSE: {mse_base:.4f}")
print(f"R²: {r2_base:.4f}")
print(f"RMSE: {np.sqrt(mse_base):.4f}")


### Aplicando Técnicas de Seleção de Features

Vamos aplicar diferentes métodos de seleção de features para identificar as mais importantes:


In [ ]:
# 1. Seleção baseada em F-score (f_regression)
f_selector = SelectKBest(score_func=f_regression, k=5)
X_train_f = f_selector.fit_transform(X_train_pima, y_train_pima)
X_test_f = f_selector.transform(X_test_pima)

# Features selecionadas pelo F-score
selected_features_f = X_pima.columns[f_selector.get_support()].tolist()
f_scores = f_selector.scores_

print("=== SELEÇÃO POR F-SCORE ===")
print(f"Features selecionadas: {selected_features_f}")
print(f"Scores F:")
for feature, score in zip(X_pima.columns, f_scores):
    print(f"  {feature}: {score:.4f}")

# 2. Seleção baseada em Mutual Information
mi_selector = SelectKBest(score_func=mutual_info_regression, k=5)
X_train_mi = mi_selector.fit_transform(X_train_pima, y_train_pima)
X_test_mi = mi_selector.transform(X_test_pima)

# Features selecionadas pelo Mutual Information
selected_features_mi = X_pima.columns[mi_selector.get_support()].tolist()
mi_scores = mi_selector.scores_

print(f"\n=== SELEÇÃO POR MUTUAL INFORMATION ===")
print(f"Features selecionadas: {selected_features_mi}")
print(f"Scores MI:")
for feature, score in zip(X_pima.columns, mi_scores):
    print(f"  {feature}: {score:.4f}")

# 3. Seleção baseada na importância do Random Forest
rf_importance = RandomForestRegressor(random_state=42, n_estimators=100)
rf_importance.fit(X_train_pima, y_train_pima)

feature_importance_rf = pd.DataFrame({
    'feature': X_pima.columns,
    'importance': rf_importance.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n=== IMPORTÂNCIA DO RANDOM FOREST ===")
print("Features ordenadas por importância:")
print(feature_importance_rf)


In [ ]:
# Comparando modelos com diferentes seleções de features

# Modelo com features selecionadas por F-score
rf_f = RandomForestRegressor(random_state=42, n_estimators=100)
rf_f.fit(X_train_f, y_train_pima)
y_pred_f = rf_f.predict(X_test_f)
mse_f = mean_squared_error(y_test_pima, y_pred_f)
r2_f = r2_score(y_test_pima, y_pred_f)

# Modelo com features selecionadas por Mutual Information
rf_mi = RandomForestRegressor(random_state=42, n_estimators=100)
rf_mi.fit(X_train_mi, y_train_pima)
y_pred_mi = rf_mi.predict(X_test_mi)
mse_mi = mean_squared_error(y_test_pima, y_pred_mi)
r2_mi = r2_score(y_test_pima, y_pred_mi)

# Modelo com top 5 features mais importantes do Random Forest
top_5_features = feature_importance_rf.head(5)['feature'].tolist()
X_train_top5 = X_train_pima[top_5_features]
X_test_top5 = X_test_pima[top_5_features]

rf_top5 = RandomForestRegressor(random_state=42, n_estimators=100)
rf_top5.fit(X_train_top5, y_train_pima)
y_pred_top5 = rf_top5.predict(X_test_top5)
mse_top5 = mean_squared_error(y_test_pima, y_pred_top5)
r2_top5 = r2_score(y_test_pima, y_pred_top5)

print("=== COMPARAÇÃO DE MODELOS ===")
print(f"{'Modelo':<25} {'MSE':<10} {'R²':<10} {'RMSE':<10}")
print("-" * 55)
print(f"{'Base (todas features)':<25} {mse_base:<10.4f} {r2_base:<10.4f} {np.sqrt(mse_base):<10.4f}")
print(f"{'F-score (top 5)':<25} {mse_f:<10.4f} {r2_f:<10.4f} {np.sqrt(mse_f):<10.4f}")
print(f"{'Mutual Info (top 5)':<25} {mse_mi:<10.4f} {r2_mi:<10.4f} {np.sqrt(mse_mi):<10.4f}")
print(f"{'RF Importance (top 5)':<25} {mse_top5:<10.4f} {r2_top5:<10.4f} {np.sqrt(mse_top5):<10.4f}")

print(f"\nTop 5 features por importância (Random Forest):")
print(top_5_features)


In [ ]:
# Visualizações das análises de seleção de features

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. F-scores
axes[0, 0].bar(X_pima.columns, f_scores)
axes[0, 0].set_title('F-Scores para Seleção de Features')
axes[0, 0].set_ylabel('F-Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Mutual Information Scores
axes[0, 1].bar(X_pima.columns, mi_scores)
axes[0, 1].set_title('Mutual Information Scores')
axes[0, 1].set_ylabel('MI Score')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Feature Importance (Random Forest)
axes[1, 0].barh(feature_importance_rf['feature'], feature_importance_rf['importance'])
axes[1, 0].set_title('Feature Importance (Random Forest)')
axes[1, 0].set_xlabel('Importance')

# 4. Comparação de Performance
models = ['Base', 'F-score', 'MI', 'RF Top5']
r2_scores = [r2_base, r2_f, r2_mi, r2_top5]
axes[1, 1].bar(models, r2_scores)
axes[1, 1].set_title('Comparação de R² Score')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].set_ylim(0, 1)

plt.tight_layout()
plt.show()


## Análise de Correlação entre Features

Vamos analisar a correlação entre as features para entender melhor as relações no dataset.


In [ ]:
# Análise de correlação
correlation_data = X_pima.copy()
correlation_data['Glucose'] = y_pima  # Adicionando a variável alvo

correlation_matrix = correlation_data.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.3f')
plt.title('Matriz de Correlação - Features vs Glucose')
plt.tight_layout()
plt.show()

# Correlação com a variável alvo (Glucose)
glucose_corr = correlation_matrix['Glucose'].drop('Glucose').sort_values(key=abs, ascending=False)
print("Correlação das features com Glucose (variável alvo):")
print(glucose_corr)


## Resumo e Insights

### Grid Search para Classificação (California Housing)

**O que aprendemos:**
- O Grid Search testou sistematicamente diferentes combinações de hiperparâmetros
- Encontrou a melhor configuração através de validação cruzada
- Melhorou o desempenho do modelo comparado aos parâmetros padrão

**Hiperparâmetros testados:**
- `max_depth`: Profundidade máxima da árvore
- `min_samples_split`: Número mínimo de amostras para dividir um nó
- `min_samples_leaf`: Número mínimo de amostras em uma folha
- `criterion`: Critério para medir a qualidade de uma divisão
- `max_features`: Número de features a considerar para a melhor divisão

### Seleção de Features para Regressão (Pima Indians Diabetes)

**Métodos aplicados:**
1. **F-Score**: Mede a relação linear entre features e variável alvo
2. **Mutual Information**: Captura dependências não-lineares
3. **Feature Importance (Random Forest)**: Baseado na redução de impureza

**Insights importantes:**
- Diferentes métodos podem selecionar features diferentes
- A seleção de features pode melhorar ou manter a performance com menos variáveis
- É importante comparar múltiplos métodos para uma análise robusta

### Próximos Passos

Para continuar explorando:
1. Testar outros algoritmos de seleção de features
2. Aplicar técnicas de redução de dimensionalidade (PCA, LDA)
3. Experimentar com diferentes números de features selecionadas
4. Usar validação cruzada para uma avaliação mais robusta


In [ ]:
# Código adicional para experimentação

# Função para testar diferentes números de features
def test_feature_selection(n_features_list, X_train, X_test, y_train, y_test):
    results = []
    
    for n_features in n_features_list:
        # Seleção por F-score
        selector = SelectKBest(score_func=f_regression, k=n_features)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)
        
        # Treinar modelo
        model = RandomForestRegressor(random_state=42, n_estimators=100)
        model.fit(X_train_selected, y_train)
        
        # Avaliar
        y_pred = model.predict(X_test_selected)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        results.append({
            'n_features': n_features,
            'r2': r2,
            'mse': mse,
            'rmse': np.sqrt(mse)
        })
    
    return pd.DataFrame(results)

# Testando diferentes números de features
n_features_list = [1, 2, 3, 4, 5, 6, 7]
results_df = test_feature_selection(n_features_list, X_train_pima, X_test_pima, 
                                   y_train_pima, y_test_pima)

print("Resultados para diferentes números de features:")
print(results_df)

# Visualizando os resultados
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(results_df['n_features'], results_df['r2'], 'o-')
plt.xlabel('Número de Features')
plt.ylabel('R² Score')
plt.title('R² Score vs Número de Features')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(results_df['n_features'], results_df['rmse'], 'o-', color='red')
plt.xlabel('Número de Features')
plt.ylabel('RMSE')
plt.title('RMSE vs Número de Features')
plt.grid(True)

plt.tight_layout()
plt.show()
